In [1]:
# import statements
import dicom
import os
import numpy as np
from skimage.draw import polygon
import matplotlib.pyplot as plt
from getContours import getContours

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
PatientID = 'UCLA_PR_5'
ROI_NUM = 0

ctFilenames = [fl for fl in os.listdir(BASE_DIR + PatientID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + PatientID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows

block_shape = (width, height, numImages) # Input to function
slice_position_z = np.zeros((numImages)).astype(np.float32) # Input to Function

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(BASE_DIR + PatientID + '/' + fl).ImagePositionPatient[2]

In [3]:
structureset = dicom.read_file(BASE_DIR + PatientID + '/structureset.dcm')
roiNumPlanes = len(structureset.ROIContourSequence[ROI_NUM].ContourSequence) 

contour_data = {} # Input to function
image_orientation = {} # Input to function
image_position = {} # Input to function
pixel_spacing = {} # Input to function

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + PatientID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing

In [4]:
contour_block, roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [8]:
positions = np.where(roi_block == 1)
print(x)

(array([122, 122, 122, ..., 333, 333, 333]), array([254, 254, 254, ..., 379, 379, 379]), array([  0,  68, 263, ...,  95, 246, 253]))


In [15]:
x_center = round(np.average(positions[0])).astype(np.uint8)
y_center = round(np.average(positions[1])).astype(np.uint8)
z_center = round(np.average(positions[2])).astype(np.uint8)
print(x_center)

244


In [16]:
centroid = [x_center, y_center, z_center]
print(centroid)

[244, 253, 158]
